# **Data Extraction**

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import sqlite3
import random

In [2]:
# List of user agents
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    # Add more user agents as needed
]

## Categories

In [3]:
# URL of the website to scrape
url = 'https://companiesmarketcap.com/all-categories/'

# Send a GET request to the URL and pass the html
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'lxml')

# Find the table containing the categories
table = soup.find('table', {'class': 'default-table table marketcap-table dataTable'})

# Extract category names and links
categories = table.find_all('a')
names = [category.text.strip() for category in categories]
links = [f"https://companiesmarketcap.com{category['href']}" for category in categories]

# Function to remove emojis from text
def remove_emojis(text):
    return re.sub(r'[^\w\s]', '', text)

# Remove emojis from category names
cleaned_names = [remove_emojis(name) for name in names]

# Create a list of tuples containing cleaned names and links
company_zip = zip(cleaned_names, links)
company_list = list(company_zip)

# Initialize an empty DataFrame to store all data
all_df = pd.DataFrame()

# Iterate over the categories
for name, url in company_list:
    # Define headers with the random user agent
    headers = {
        'User-Agent': random.choice(user_agents),
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',}
    
    # Send a GET request to the category URL
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')

    # Extract statistics from the category stats bar
    stat_bar = soup.find('div', {'class': 'category-stats-bar'})
    stats = stat_bar.find_all('span', {'class': 'font-weight-bold'})
    company_no = int(stats[0].text.replace(',', ''))
    total_cap = stats[-1].text.strip()

    # Extract company names from the table
    table = soup.find('tbody')
    names = table.find_all('div', {'class': 'company-name'})
    names = [name.text.strip() for name in names]

    # Extract earnings and share prices from the table
    tds = table.find_all('td', {'class': 'td-right'})
    earnings = []
    prices = []
    for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        price_text = tds[i+2].text.replace('$', '').replace(',', '')
        if price_text == 'N/A':
            prices.append(None)
        else:
            prices.append(float(price_text))

    # Extract country names
    countries = soup.find_all('span', {'class': 'responsive-hidden'})
    countries = countries[1:]
    countries = [country.text.strip() for country in countries]

    # Function to convert values to billions
    def convert_to_billion(value):
        if value == 'N/A':
            return None  # or return 0 if you prefer
        if 'T' in value:
            return float(value.replace('T', '')) * 1000
        elif 'B' in value:
            return float(value.replace('B', ''))
        elif 'M' in value:
            return float(value.replace('M', '')) / 1000
        else:
            return float(value)

    # Create a DataFrame for the current category
    data = {
        'Company': names,
        'Share Price($)': prices,
        'Country': countries,
        'Total Cap($B)': total_cap,
        'Number of Companies': company_no
    }
    df = pd.DataFrame(data)

    # Clean and convert total cap values and add category column
    df['Total Cap($B)'] = df['Total Cap($B)'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)
    df['Category'] = name

    # Concatenate the current category DataFrame with the main DataFrame
    all_df = pd.concat([all_df, df], ignore_index=True)

# Add a timestamp column to the DataFrame
all_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

# Display the final DataFrame info
all_df.info()

C:\Users\Eliud\AppData\Local\Temp\ipykernel_10136\2282115141.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_df = pd.concat([all_df, df], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7239 entries, 0 to 7238
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company              7239 non-null   object 
 1   Share Price($)       7235 non-null   float64
 2   Country              7239 non-null   object 
 3   Total Cap($B)        7239 non-null   float64
 4   Number of Companies  7239 non-null   int64  
 5   Category             7239 non-null   object 
 6   As Of                7239 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 396.0+ KB


In [4]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Add data to table
all_df.to_sql('Categories', conn, if_exists='replace', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Assets

In [5]:
url = 'https://companiesmarketcap.com/assets-by-market-cap/'
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'lxml')
table = soup.find('table', {'class':'default-table table marketcap-table dataTable'})


entries = table.find_all('tr')
entries = entries[1:]

names = []
trends =[]
pct_change = []
m_cap = []
prices = []
links = []
asset_desc = []

for entry in entries:
    name = entry.find('div', {'class':'company-name'})
    names.append(name.text.strip())

    earn = entry.find('td', {'class':'rh-sm'})
    trend = 'down' if earn.find('span', {'class': 'percentage-red'}) else 'up'
    trends.append(trend)
    
    day_ct = earn.text
    pct_change.append(day_ct)

    fintext = entry.find_all('td', {'class':'td-right'})
    cap = fintext[1].text.strip()
    m_cap.append(cap)
    
    price = fintext[2].text.strip()
    prices.append(price)
    

    url = entry.find('a')
    if url:
        link = url['href']
        full_link = f'https://companiesmarketcap.com{link}'
    else:
        full_link = None

    links.append(full_link)

for link in links:
    if link:
        
        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',}

        r = requests.get(link, headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')
        
        div = soup.find('div', {'class': 'col-sm-9'})
        if not div:
            div = soup.find('div', {'class': 'col-lg-4 company-description'})
        
        text = div.text.strip() if div else None
        asset_desc.append(text)
    else:
        asset_desc.append(None)      
        
# Save to dataframe        
asset_df = pd.DataFrame({'Asset': names, 'Market Cap': m_cap, 'Share Price': prices,
                   'Day Change': pct_change, 'Trend': trends,  'Description': asset_desc
                   })


# Data cleaning

# Function to convert values to billions
def convert_to_billion(value):
    if 'T' in value:
        return float(value.replace('T', '')) * 1000
    elif 'B' in value:
        return float(value.replace('B', ''))
    else:
        return float(value)

# Apply the conversion function and strip '$' and 'T'
asset_df['Market Cap'] = asset_df['Market Cap'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

# Rename the column
asset_df = asset_df.rename(columns={'Market Cap': 'Market Cap($B)'})
asset_df['Share Price'] = asset_df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
asset_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')
asset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Asset           100 non-null    object 
 1   Market Cap($B)  100 non-null    float64
 2   Share Price     100 non-null    float64
 3   Day Change      100 non-null    object 
 4   Trend           100 non-null    object 
 5   Description     58 non-null     object 
 6   As Of           100 non-null    object 
dtypes: float64(2), object(5)
memory usage: 5.6+ KB


In [6]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Update table
asset_df.to_sql('Assets', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Earnings

In [7]:
# Generate a list of URLs for the first 10 pages of the most profitable companies

urls = []
base = 'https://companiesmarketcap.com/most-profitable-companies/page/'
for pg in range(1, 11):
  url = f'{base}{pg}'
  urls.append(url)


In [8]:
all_df = []
for url in urls:
  # Define headers with the random user agent
  headers = {
      'User-Agent': random.choice(user_agents),
      'Accept-Language': 'en-US,en;q=0.5',
      'Connection': 'keep-alive',}

  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text.strip() for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  earnings = []
  prices = []
  for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text.strip() for country in countries]

  data = {'Company':names, 'Earnings':earnings, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  # Function to convert values to billions
  def convert_to_billion(value):
      if 'T' in value:
          return float(value.replace('T', '')) * 1000
      elif 'B' in value:
          return float(value.replace('B', ''))
      else:
          return float(value)

  # Apply the conversion function and strip '$' and 'T'
  df['Earnings'] = df['Earnings'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

  # Rename the column
  df = df.rename(columns={'Earnings': 'Earnings($B)'})
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
earnings_df = pd.concat(all_df, ignore_index=True)

# Drop duplicates 
earnings_df.drop_duplicates(subset='Company', keep='first', inplace=True)
earnings_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

# Sort alphabetically and reset index
earnings_df = earnings_df.sort_values(by='Company')
earnings_df = earnings_df.reset_index(drop=True)
earnings_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

earnings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       1000 non-null   object 
 1   Earnings($B)  1000 non-null   float64
 2   Share Price   1000 non-null   float64
 3   Pct_change    1000 non-null   float64
 4   Trend         1000 non-null   object 
 5   Country       1000 non-null   object 
 6   As Of         1000 non-null   object 
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [9]:
earnings_df.head()

,Company,Earnings($B),Share Price,Pct_change,Trend,Country,As Of
0,3i Group,5.66,39.60,1.92,up,UK,2024-07-29 16:17
1,7-Eleven,1.67,11.55,2.63,up,Japan,2024-07-29 16:17
2,ABB,4.84,54.29,0.41,down,Switzerland,2024-07-29 16:17
3,ABN AMRO,4.05,17.75,0.27,down,Netherlands,2024-07-29 16:17
4,AES,1.46,17.52,1.68,up,USA,2024-07-29 16:17


In [10]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Update table
earnings_df.to_sql('Earnings', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Market cap

In [11]:
# Generate urls for companies with the highest market caps

base = 'https://companiesmarketcap.com/page/'
urls = []
for pg in range(1,11):
  url = f'{base}{pg}'
  urls.append(url)

In [12]:
all_df = []
for url in urls:
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text.strip() for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  earnings = []
  prices = []
  for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text.strip() for country in countries]

  data = {'Company':names, 'Market cap':earnings, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  # Function to convert values to billions
  def convert_to_billion(value):
      if 'T' in value:
          return float(value.replace('T', '')) * 1000
      elif 'B' in value:
          return float(value.replace('B', ''))
      else:
          return float(value)

  # Apply the conversion function and strip '$' and 'T'
  df['Market cap'] = df['Market cap'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

  # Rename the column
  df = df.rename(columns={'Market cap': 'Market cap($B)'})
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
market_df = pd.concat(all_df, ignore_index=True)
market_df.drop_duplicates(subset='Company', keep='first', inplace=True)

# Sort alphabetically and reset index
market_df = market_df.sort_values(by='Company')
market_df = market_df.reset_index(drop=True)
market_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         1000 non-null   object 
 1   Market cap($B)  1000 non-null   float64
 2   Share Price     1000 non-null   float64
 3   Pct_change      1000 non-null   float64
 4   Trend           1000 non-null   object 
 5   Country         1000 non-null   object 
 6   As Of           1000 non-null   object 
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [13]:
market_df.head()

,Company,Market cap($B),Share Price,Pct_change,Trend,Country,As Of
0,3M,70.36,127.16,22.99,up,USA,2024-07-29 16:18
1,3i Group,38.18,39.60,1.92,up,UK,2024-07-29 16:18
2,7-Eleven,29.97,11.55,2.63,up,Japan,2024-07-29 16:18
3,ABB,100.32,54.29,0.41,down,Switzerland,2024-07-29 16:18
4,ABB India,20.18,95.27,1.57,up,India,2024-07-29 16:18


In [14]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Update table
market_df.to_sql('Marketcap', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Revenue

In [15]:
base = 'https://companiesmarketcap.com/largest-companies-by-revenue/page/'
urls = []
for pg in range(1,11):
  url = f'{base}{pg}'
  urls.append(url)

In [16]:
all_df = []
for url in urls:
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  earnings = []
  prices = []
  for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text for country in countries]

  data = {'Company':names, 'Revenue':earnings, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  # Function to convert values to billions
  def convert_to_billion(value):
      if 'T' in value:
          return float(value.replace('T', '')) * 1000
      elif 'B' in value:
          return float(value.replace('B', ''))
      else:
          return float(value)

  # Apply the conversion function and strip '$' and 'T'
  df['Revenue'] = df['Revenue'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

  # Rename the column
  df = df.rename(columns={'Revenue': 'Revenue($B)'})
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
revenue_df = pd.concat(all_df, ignore_index=True)

revenue_df.drop_duplicates(subset='Company', keep='first', inplace=True)

# Sort alphabetically and reset index
revenue_df = revenue_df.sort_values(by='Company')
revenue_df = revenue_df.reset_index(drop=True)
revenue_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1000 non-null   object 
 1   Revenue($B)  1000 non-null   float64
 2   Share Price  1000 non-null   float64
 3   Pct_change   1000 non-null   float64
 4   Trend        1000 non-null   object 
 5   Country      1000 non-null   object 
 6   As Of        1000 non-null   object 
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [17]:
revenue_df.head(10)

,Company,Revenue($B),Share Price,Pct_change,Trend,Country,As Of
0,3M,32.65,127.16,22.99,up,USA,2024-07-29 16:18
1,7-Eleven,77.07,11.55,2.63,up,Japan,2024-07-29 16:18
2,A2A,18.24,2.15,0.46,up,Italy,2024-07-29 16:18
3,ABB,32.24,54.22,0.54,down,Switzerland,2024-07-29 16:18
4,AECOM,15.34,90.23,1.44,up,USA,2024-07-29 16:18
5,AEGON\n,12.01,6.35,1.60,up,Netherlands,2024-07-29 16:18
6,AES,12.51,17.52,1.68,up,USA,2024-07-29 16:18
7,AGC,13.85,34.16,1.45,up,Japan,2024-07-29 16:18
8,AGCO,14.00,103.52,1.65,up,USA,2024-07-29 16:18
9,AIA,20.69,6.72,1.55,up,Hong Kong,2024-07-29 16:18


In [18]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Update table
revenue_df.to_sql('Revenue', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Employee size

In [19]:
base = 'https://companiesmarketcap.com/largest-companies-by-number-of-employees/page/'
urls = []
for pg in range(1,11):
  url = f'{base}{pg}'
  urls.append(url)

In [20]:
all_df = []
for url in urls:
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text.strip() for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  employees = []
  prices = []
  for i in range(0, len(tds), 3):
        employees.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text.strip() for country in countries]

  data = {'Company':names, 'Employees':employees, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  df['Employees'] = df['Employees'].str.replace(r'[,]', '', regex=True).astype(int)
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
employee_df = pd.concat(all_df, ignore_index=True)

employee_df.drop_duplicates(subset='Company', keep='first', inplace=True)

# Sort alphabetically and reset index
employee_df = employee_df.sort_values(by='Company')
employee_df = employee_df.reset_index(drop=True)
employee_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

employee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1000 non-null   object 
 1   Employees    1000 non-null   int64  
 2   Share Price  1000 non-null   float64
 3   Pct_change   1000 non-null   float64
 4   Trend        1000 non-null   object 
 5   Country      1000 non-null   object 
 6   As Of        1000 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB


In [21]:
employee_df.head(10)

,Company,Employees,Share Price,Pct_change,Trend,Country,As Of
0,3M,85000,127.16,22.99,up,USA,2024-07-29 16:19
1,7-Eleven,84154,11.55,2.63,up,Japan,2024-07-29 16:19
2,ABB,108700,54.22,0.54,down,Switzerland,2024-07-29 16:19
3,ABM Industries,100000,55.08,0.86,up,USA,2024-07-29 16:19
4,AECOM,52000,90.23,1.44,up,USA,2024-07-29 16:19
5,AGC,56724,34.16,1.45,up,Japan,2024-07-29 16:19
6,AGCO,27900,103.52,1.65,up,USA,2024-07-29 16:19
7,AIER Eye Hospital,32326,1.40,1.74,down,China,2024-07-29 16:19
8,ALSOK,39039,6.15,1.05,up,Japan,2024-07-29 16:19
9,ALTEN,57000,108.93,1.08,down,France,2024-07-29 16:19


In [22]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Update table
employee_df.to_sql('Employees', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


# **Transformation**

In [23]:
# Select only the relevant columns from each DataFrame
earns = earnings_df[['Company', 'Earnings($B)', 'Country', 'Trend']]
cap = market_df[['Company', 'Market cap($B)']]
revenue = revenue_df[['Company', 'Revenue($B)']]
employee = employee_df[['Company', 'Employees', 'Share Price']]

# Merge the DataFrames on the 'Company' column
merged_df = pd.merge(earns, cap, on='Company', suffixes=('_earn', '_cap'))
merged_df = pd.merge(merged_df, revenue, on='Company', suffixes=('', '_rev'))
merged_df = pd.merge(merged_df, employee, on='Company', suffixes=('', '_emp'))

# Reorder columns
cols = merged_df.columns.tolist()
cols.remove('Trend')
cols.remove('Country')  
cols.append('Country')
cols.append('Trend')
merged_df = merged_df[cols] 
merged_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         337 non-null    object 
 1   Earnings($B)    337 non-null    float64
 2   Market cap($B)  337 non-null    float64
 3   Revenue($B)     337 non-null    float64
 4   Employees       337 non-null    int64  
 5   Share Price     337 non-null    float64
 6   Country         337 non-null    object 
 7   Trend           337 non-null    object 
 8   As Of           337 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 23.8+ KB


In [24]:
merged_df.head(10)

,Company,Earnings($B),Market cap($B),Revenue($B),Employees,Share Price,Country,Trend,As Of
0,7-Eleven,1.67,29.97,77.07,84154,11.55,Japan,up,2024-07-29 16:19
1,ABB,4.84,100.32,32.24,108700,54.22,Switzerland,down,2024-07-29 16:19
2,ANZ Bank,6.48,57.34,13.05,40342,18.88,Australia,down,2024-07-29 16:19
3,ASML,8.63,349.85,27.46,40940,888.39,Netherlands,up,2024-07-29 16:19
4,AT&T,18.95,136.30,122.31,148290,19.01,USA,down,2024-07-29 16:19
5,AXA,7.36,78.92,134.68,94705,35.03,France,down,2024-07-29 16:19
6,AbbVie,7.53,326.96,54.40,50000,185.16,USA,up,2024-07-29 16:19
7,Abbott Laboratories,6.53,183.07,40.32,114000,105.24,USA,down,2024-07-29 16:19
8,Accenture,9.37,205.74,64.47,750000,328.46,Ireland,down,2024-07-29 16:19
9,Adobe,6.57,240.51,20.42,29945,542.44,USA,up,2024-07-29 16:19


In [25]:
# Save to SQLite Database
conn = sqlite3.connect('data/Marketcap.db')
cursor = conn.cursor()

# Update table
merged_df.to_sql('CompanyMerge', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


# **Insight Extraction**

### Explore top companies

In [ ]:
metrics = cols[1:-2]

print(f'Sample metrics include:')
for i, metric in enumerate(metrics, start=1):
    print(f"{i}. {metric}")

In [ ]:
# Sort and display dataframe
sort_key = input('Enter a metric to sort by:')
choice = metrics[int(sort_key)-1]
sort_df = merged_df.sort_values(by=choice, ascending=False)
sort_df = sort_df.reset_index(drop=True)
print(f'\nDisplaying top companies by {choice}:\n')
sort_df.head(20)

### Explore trends

In [ ]:
trend = input('Select a trend; up or down:')

filtered_df = merged_df[merged_df['Trend'] == trend]
df_size = len(filtered_df)

if trend == 'up':
  if df_size > 0:
    print(f"There are {len(filtered_df)} Companies with increasing trends today:")
    print(filtered_df['Company'].head(30))
  else:
    print("No companies found with increasing trends.")
elif trend =='down':
  if df_size > 0:
    print(f"There are {len(filtered_df)} Companies with decreasing trends today:")
    filtered_df.head(30)
  else:
    print("No companies found with decreasing trends.")
else:
  print("Invalid input. Please try again.") 
